In [ ]:
import pandas as pd
import os
import api
import re

default = r'C:\Users\AI\Desktop\일본'
region_list = os.listdir(r'C:\Users\AI\Desktop\일본')

for region in region_list:
    os.chdir(default)
    os.chdir(region + r'\info')
    
    if 'restaurant_scraped_list_final.csv' in os.listdir():
        print(region + '지역은 이미 파일이 존재하므로 다음 지역으로 넘어갑니다.')
        continue
    try:
        info = pd.read_csv('restaurant_scraped_list.csv')
    except FileNotFoundError:
        continue
        
    #컬럼 재정의
    info.columns = ['index','title','all_score','food_score','service_score','price_score'
                    ,'feel_score','price','price_count','food','special_food','meal_time','special'
                    ,'call','working_day','working_time','address','lat','lng']    
    del info['index']
    
    #필터링 (스코어0인것 제외)
    info = info[(info['food_score'] != 0) & (info['service_score'] != 0) & (info['price_score'] != 0) & (info['feel_score'] != 0)]
    for i in range(0,len(info)):
        #타이틀 소문자화
        info['title'].iloc[i] = info['title'].iloc[i].lower()
        #문자 # 없애기
        info['address'].iloc[i] = re.sub('[#]','',info['address'].iloc[i])
        #레스토랑 이름이 같은것 중복 제거
        info.drop_duplicates('title')
        #주소로 api에서 위경도 받아오기
        info['lat'].iloc[i],info['lng'].iloc[i] = api.getLatLng(info['address'].iloc[i],info['title'].iloc[i],region,i)
    
    #위경도 제대로 안나온거 (0,0) 제외
    info = info[(info['lat'] != '0.0') & (info['lng'] != '0.0')]
    #다시 저장
    info.to_csv('restaurant_scraped_list_final.csv',index=False,encoding='utf-8')


In [7]:
import pandas as pd

data = pd.read_csv(r'C:\Users\AI\Desktop\japan\hokkaido\info\restaurant_scraped_list_final.csv')

In [8]:
data.drop_duplicates('address')
data['region'] = 'hokkaido'
data.to_csv()

,title,all_score,food_score,service_score,price_score,feel_score,price,price_count,food,special_food,meal_time,special,call,working_day,working_time,address,lat,lng
0,toriton kaitensushi yuhigaokaten,4.5,4.5,4.0,4.0,4.0,₩₩ - ₩₩₩,,"일본 요리, 스시",,"점심식사, 저녁식사","테이크아웃, 예약",+81 157-31-8311,['일 - 토'],['오전 11:00 - 오후 10:00'],"2-5-1 Miyoshicho, Kitami, Hokkaido",43.809241,143.878058
1,yojo hormone,4.0,4.0,4.0,4.0,3.5,₩₩ - ₩₩₩,,바베큐,,저녁식사,,+81 157-23-1927,['일 - 토'],['오후 5:00 - 오후 11:00'],"1-14 Kita 4-jo Nishi | Tekkousen Hall 2F, Kita...",43.803955,143.891166
2,numanoya,4.0,4.0,3.5,4.0,4.0,₩,,,,,테이크아웃,+81 138-67-2104,['일 - 토'],['오전 8:30 - 오후 6:00'],"145 Onumacho, Nanae-cho, Kameda-gun 041-1354, ...",41.968251,140.680508
3,pastry snaffle's kanamori yobutsukan,4.0,4.0,4.0,4.0,4.5,₩₩ - ₩₩₩,,카페,,,"테이크아웃, 휠체어 이용 가능, 예약",+81 138-27-1240,['일 - 토'],['오전 9:30 - 오후 7:00'],"13-9 Suehirocho, Hakodate 040-0053, Hokkaido",41.766723,140.716587
4,"pastry snaffle's, takaoka",4.5,4.5,4.0,4.0,3.5,₩₩ - ₩₩₩,"₩10,539 - ₩32,787",,,,예약,+81 138-59-6660,['목 - 화'],['오전 11:00 - 오후 6:00'],"178-1 Takaokacho, Hakodate 042-0955, Hokkaido",41.784799,140.808194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576,hasegawa store yunogawa,4.0,4.0,3.0,4.0,3.0,일본 요리,,일본 요리,,,,+81 138-57-2400,,,"1-10-15, Yunokawa, Hakodate, Hokkaido",41.774475,140.784260
1577,restaurant pollux,3.5,3.5,3.5,3.0,3.5,인터내셔널,,"인터내셔널, 일본 요리",,"아침식사, 점심식사, 저녁식사",,+81 138-57-8884,['일 - 토'],['오전 8:00 - 오후 7:30'],"511 Takamatsucho | Hakodate Airport, Hakodate ...",41.775704,140.815822
1578,yamakawabokujo,4.0,4.0,3.5,3.5,3.5,₩₩ - ₩₩₩,,,,음료,테이크아웃,+81 138-67-2114,,,"628 Onumacho, Nanae-cho, Kameda-gun 041-1354, ...",41.968251,140.680508
1579,starbucks coffee hakodate bayside,4.0,3.5,4.0,3.5,4.5,₩₩ - ₩₩₩,,카페,채식주의 식단,"아침식사, 점심식사, 브런치, 음료","테이크아웃, 좌석, 야외석",+81 138-21-4522,['일 - 토'],['오전 8:00 - 오후 11:00'],"24-6 Suehirocho | Hakodate Nishi Wharf, Hakoda...",41.765539,140.715074
